# Table of contents
TODO: titles with internal links to move there

# 1) Introduction
Group members:
- Eugene Phang (EPHANG002)

# 2) Decomposition and breakdown

blah blah

# 3) Analysis of data given (records.csv)

# Extracting Data from CSV

file = original excel file
rawFileData = giant list where each item is a row converted into a single string
fileHeader = first row "Tutorial Group,Student ID,School,Name,Gender,CGPA"
fileData = each row in the file turned into a list of cells as strings
studentData = list of dictionaries, each representing a student's data ('student ID', 'TG', 'School', 'Name', 'Gender', 'CGPA')

In [29]:
file = open("records.csv", "r")
rawFileData = file.readlines()
fileData = []
for i in rawFileData:
    i = i.split(",")
    i[5] = i[5].replace('\n', '')
    fileData.append(i)
fileHeader = fileData.pop(0) #list of cells from first line

print(fileData[0])

['G-1', '5002', 'CCDS', 'Aarav Singh', 'Male', '4.02']


In [4]:
#global variables

In [5]:
def initStudents():      #creates studentData, a list of dictionaries, each representing a student
    studentData = []
    studentCount = 0
    for row in fileData:
        studentCount += 1
        if row[4] == 'Male':
            gender = True
        elif row[4] == 'Female':
            gender = False
        studentData.append({'ID': int(row[1]),'TG': int(row[0][2:]), 'School': row[2], 'Name': row[3], 'Gender': gender, 'CGPA': float(row[5])})
    return studentData

In [6]:
studentData = initStudents()
print(studentData[0])
print(len(studentData))

{'ID': 5002, 'TG': 1, 'School': 'CCDS', 'Name': 'Aarav Singh', 'Gender': True, 'CGPA': 4.02}
6000


# 4) Design of Algorithm

## 4.1) Sorting Priorities
1. School
2. CGPA
3. Gender

We decided on this priority list based on what most strongly influences group dynamics / learning outcomes.

School was placed as the highest priority because we believe that the merit of a group project is the opportunity to experience and work with a diversity of perspectives that you wouldn't otherwise experience in a solo project. Students from different schools provide unique educational backgrounds and problem solving methods which encourages balanced and high quality work. 

CGPA is second priority as it is an indicator of academic performance and work consistency. Prioritising CGPAis crucial to achieve a balanced mix of abilities and ensure that no single group is disproportionately strong or weak academically. However, CGPA is still placed as a lower priority because we believe that CGPA is not wholly representative of one's abilities.

Gender was placed last as a final consideration to maintain inclusivity and promote balanced representation within each group. It acts as a guiding constraint in the algorithm. 

## 4.2) Group Sizes
Currently, we are given a simplified version of the problem in which the desired number of students per group ($n$) is $5$ and the existing dataset contains a total of $6000$ students. Since $5|6000$, we can safely make $\frac{6000}{5}=1200$ equal groups of $5$. However, in an extension of this problem, the professor may specify $n$ that does not exactly divide the total number of students in a given dataset.

In this case, we decided to handle the remainder students ($r$) by distributing them, one to each existing group, with the condition that the number of groups with $n$ students is greater than the number of groups with $n+1$ students. In other words, we distribute the students $\text{if }r<\frac{total}{2n}$ (Case 1), else create a separate group of size $r$ (Case 2).

This condition is to ensure that there doesn't arise a case where the professor asks for groups of $n$ but there are finally more groups of $n+1$ than there are groups of $n$ produced by the algorithm. 

The code written below demonstrates how this is implemented:

In [7]:
def createGrps(studentData, n):
    total = len(studentData) # Total number of students 
    r = total % n # Remainder of students when the population is divided into groups of n
    numGrp = total // n if (r <= total/(2 * n)) else total // n + 1 # Number of final groups to be created
        
    # Deciding on the size of each group based on the rules specified
    grpSizes =  [n + 1 if i < r else n for i in range(numGrp)] if (r <= total/(2 * n)) else [r if i == total//n else n for i in range(numGrp)]
    print(f"{grpSizes} Size {len(grpSizes)}")

createGrps(studentData, 100) # Example: Simplified Case - n exactly divides total
createGrps(studentData, 34) # Example: Case 1 
createGrps(studentData, 134) # Example: Case 2 
createGrps(studentData, 261) # Example: Case 2 

[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100] Size 60
[35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34

## 4.3) Brainstorming: Individual Modular Diversification of Factors

Before arriving at a final solution, let us decompose the algorithm into its three constituent sections: diversification by CGPA, diversification by school, and diversification by gender. For this, we may come up with individual algorithms / rules to diversify each factor, and later combine the three to arrive at the final algorithm such that the priority order specified above is obeyed. 

### 4.3.1) Considering School Frequency: Diversifying School
To ensure that people in all schools are distributed diversely, we must distribute students in the order of the frequency of their respective schools in the population dataset. In other words, students belonging to schools with a higher number of representatives are distributed first, and those from smaller schools are assigned later.

This approach is logical because schools with larger student counts have a higher likelihood of forming clusters if they are left unregulated. By prioritizing their distribution early, we prevent these large groups from dominating the later stages of assignment, where additional lower priority constraints that are unrelated to school affiliation will take effect. At that point, having students from smaller schools remaining is less risky to diversity than if a large concentration of students from the same school were left unassigned.

The code written below demonstrates how this is implemented. Note that the code discards the other two factors and soley diversifies groups based on the students' schools.

In [8]:
def orderSchool (studentData):
    # Create a sorted list of schools in descending order of population
    schoolCnt = {}
    for student in studentData:
        schoolCnt[student['School']] = schoolCnt.get(student['School'], 0) + 1
    sortedSchools = sorted(schoolCnt.items(), key=lambda x: x[1], reverse=True)

    # Sort students based on the index of their school in sortedSchools 
    sortedStudents = sorted(studentData, key=lambda s: next(i for i, (school, _) in enumerate(sortedSchools) if school == s['School']))

    return [sortedSchools, sortedStudents]

orderedSchool = orderSchool(studentData)
print(orderSchool(studentData)[0], "\n")
for student in orderedSchool[1]:
    print(f"{student['Name']} {student['School']}")

[('CoB (NBS)', 793), ('EEE', 788), ('CCDS', 702), ('MAE', 512), ('CCEB', 465), ('SSS', 425), ('SoH', 332), ('CEE', 321), ('SBS', 302), ('SPMS', 276), ('CoE', 275), ('NIE', 199), ('MSE', 156), ('WKW SCI', 150), ('ADM', 116), ('ASE', 104), ('LKCMedicine', 83), ('HASS', 1)] 

Ajay Verma CoB (NBS)
Areeba Khan CoB (NBS)
Felix Yip CoB (NBS)
Harlow Wang CoB (NBS)
Isabella Thompson CoB (NBS)
Nisha Das CoB (NBS)
Oliver Tan CoB (NBS)
Vivi Dwi CoB (NBS)
Christopher Miller CoB (NBS)
Leo Xu CoB (NBS)
Michael Miller CoB (NBS)
Paula Wong CoB (NBS)
Alina Malik CoB (NBS)
Ameer Bin Rahman CoB (NBS)
Fangling CoB (NBS)
Fergus Mendoza CoB (NBS)
Han Zhang CoB (NBS)
Maya Ong CoB (NBS)
Siti Maryam CoB (NBS)
Yumi Nishikawa CoB (NBS)
Jin Yang Tan CoB (NBS)
Madiha Ali CoB (NBS)
Rachel Choi CoB (NBS)
Sophia Jones CoB (NBS)
Suraj Mishra CoB (NBS)
Gina Lau CoB (NBS)
Kanwal Bukhari CoB (NBS)
Nisha Gupta CoB (NBS)
Chong Wang CoB (NBS)
Samantha Khan CoB (NBS)
Selina Loh CoB (NBS)
Simon Lee CoB (NBS)
Zachary Lee CoB (N

In [35]:
def sortSchool(studentData, n):
    
    total = len(studentData) 
    r = total % n
    numGrp = total // n if (r <= total/(2 * n)) else total // n + 1
    grps = [[] for _ in range(numGrp)]

    students = orderSchool(studentData)[1]
    
    for i, student in enumerate(students): 
        grps[i % numGrp].append(student) # Round-robin distribution of sorted list

    return grps

grps = sortSchool(studentData, 7)
for i, g in enumerate(grps, start=1):
    print(f"Group {i}")
    for s in g:
        print(f"  - {s['Name']}, {s['School']}")

Group 1
  - Ajay Verma, CoB (NBS)
  - Harini Nair, EEE
  - Nash Tan, CCDS
  - Anu Rao, MAE
  - Atsuko Taniguchi, SSS
  - Ronin Smith, CEE
  - Hana Ng, CoE
  - Dorian Yang, HASS
Group 2
  - Areeba Khan, CoB (NBS)
  - Maya Devi, EEE
  - Nitesh Singh, CCDS
  - Cahya Sari, MAE
  - Elijah Wong, SSS
  - Toni Saputra, CEE
  - Jing Yi Tan, CoE
Group 3
  - Felix Yip, CoB (NBS)
  - Shoji Takahashi, EEE
  - Vesper Salazar, CCDS
  - Leah Liu, MAE
  - Jasmine Ng, SSS
  - Haruto Yamamoto, CEE
  - Marwan Bin Ali, CoE
Group 4
  - Harlow Wang, CoB (NBS)
  - Shreya Rani, EEE
  - Yui Yamamoto, CCDS
  - Theo Campbell, MAE
  - Roxanne Chan, SSS
  - Ajay Singh, CEE
  - Nathan Tan, CoE
Group 5
  - Isabella Thompson, CoB (NBS)
  - Tara Lim, EEE
  - Zhonghua, CCDS
  - Elijah Scott, MAE
  - Sania Malik, SSS
  - Emma Bryant, CEE
  - Ravi Deshmukh, CoE
Group 6
  - Nisha Das, CoB (NBS)
  - Zheng Guo, EEE
  - Ashok Patil, CCDS
  - Penny Yip, MAE
  - Sofia Ramirez, SSS
  - Gabriel Morris, CEE
  - Arjun Patel, CoE
Gr

### 4.3.2) Buckets & Cyclic Round-Robin: Diversifying CGPA
To ensure that each group includes a range of CGPAs, we can implement a kind of round-robin distribution. 

To do this, we first sort the group of students in descending order of CGPA, and partition them into approximately homogeneous GPA "buckets" so that we can intuitively label them as ex. "low-GPA", "mid-GPA", "high-GPA", etc. 

Then, the students in each bucket are assigned to groups in a round-robin fashion. If we were to distribute students sequentially within each bucket (i.e., the first student always goes to Group 1, the second to Group 2, and so on), the first group would consistently receive the top students from each bucket. This may result in an unfair imbalance wherein the first group has a noticeably higher average CGPA than, say, the last. To avoid this, we can implement a cyclic distribution on top of the round robin in which the starting index for group assignment shifts each time a new bucket is processed, which ensures that no group consistently receives students from the same position within each bucket.

To visualise, suppose each space separates buckets and each number indicates the group a student is assigned to:
- Without cycling: 0123 0123 0123 0123
- With cycling: 0123 1230 2301 3012

The code written below demonstrates how this is implemented. Note that the code discards the other two factors and soley diversifies groups based on the students' CGPAs.

In [10]:
def GPABucket (studentData, k): # Arbitrary value k to decide size of bucket
    # Sort list in descending order of CGPA
    studentData.sort(key=lambda x: x['CGPA'], reverse=True)
    partition = [studentData[i:i + k] for i in range(0, len(studentData), k)] # Create buckets of size n
    return partition

# Printing buckets in order
partition = GPABucket(studentData, 6)
for index, bucket in enumerate(partition, start=1):
        print(f"\nBucket {index} ({len(bucket)}):")
        for student in bucket:
            print(f"{student['Name']} {student['CGPA']}", end=" ")


Bucket 1 (6):
Hunter Green 4.97 Dante Ong 4.97 Nadia Binti Zain 4.97 Benjamin Hernandez 4.79 Suhana Bhat 4.79 Aira Binti 4.79 
Bucket 2 (6):
Ciara Lim 4.79 Claire Ng 4.79 Sari Rakhmawati 4.79 Maira Binti Rahman 4.73 Yanzhi 4.73 Liyana Binti Mohd 4.73 
Bucket 3 (6):
Yongqiang 4.73 Nabihah Binti Rahman 4.73 Laras Wati 4.73 Matthew Martinez 4.57 Faisal Bin Yusof 4.57 Ravi Mehta 4.57 
Bucket 4 (6):
Layla Torres 4.52 Hae-Jin Lee 4.52 Yasir Ahmad 4.52 Ashok Nair 4.52 Zadie Harris 4.52 Asha Mehta 4.52 
Bucket 5 (6):
Jason Robinson 4.52 Asher Thompson 4.52 Mateo Huang 4.52 Layla Reddy 4.52 Troy Wong 4.52 Nicolas Ramos 4.52 
Bucket 6 (6):
Uriah Howard 4.52 Harper Martinez 4.52 Nitesh Singh 4.52 Fauzi Bin Zain 4.52 Samantha Lee 4.52 Chandra Widyanto 4.52 
Bucket 7 (6):
Aaron Lee 4.52 Priyanka Singh 4.52 Olivia Johnson 4.52 Sadia Khan 4.52 Mi-kyung Han 4.52 Fatima Reddy 4.52 
Bucket 8 (6):
Luna Zhao 4.52 Jin Wei 4.52 Sophia Johnson 4.52 Tristan Smith 4.52 Lola Li 4.52 Tam Nguyen 4.52 
Bucket 9 (

In [11]:
def sortCGPA(studentData, n):

    total = len(studentData) 
    r = total % n
    numGrp = total // n if (r <= total/(2 * n)) else total // n + 1
    grpSizes =  [n + 1 if i < r else n for i in range(numGrp)] if (r <= total/(2 * n)) else [r if i == total//n else n for i in range(numGrp)]
    grps = [[] for _ in range(numGrp)]

    partition = GPABucket(studentData, numGrp)
    
    grpIndex = 0
    for bucketInd, bucket in enumerate(partition):
        startInd = bucketInd % numGrp
        for student in bucket:
            grps[(startInd + grpIndex) % numGrp].append(student)
            grpIndex += 1
    return grps

grps = sortCGPA(studentData, 5)
for i, g in enumerate(grps, start=1):
    print(f"Group {i}")
    for s in g:
        print(f"  - {s['Name']}, {s['CGPA']}")

Group 1
  - Hunter Green, 4.97
  - Layla Zhang, 4.12
  - Dallas Liu, 4.05
  - Rishi Sharma, 4.01
  - Bharat Menon, 3.78
Group 2
  - Dante Ong, 4.97
  - Xing Ying, 4.18
  - Ronald Tan, 4.05
  - Stella James, 4.01
  - Mohammed Sadiq, 3.75
Group 3
  - Nadia Binti Zain, 4.97
  - Megan Tsai, 4.18
  - Mason Reyes, 4.12
  - Kendall Yang, 4.01
  - Fatimah Binti Ahmad, 3.75
Group 4
  - Benjamin Hernandez, 4.79
  - Sophie Zhao, 4.18
  - Anil Reddy, 4.12
  - Kathy Tan, 4.05
  - Rachael Tan, 3.7
Group 5
  - Suhana Bhat, 4.79
  - Stepan Aleksandrov, 4.18
  - Ava Nguyen, 4.12
  - Zedrick Ong, 4.05
  - Ryo Tanaka, 4.01
Group 6
  - Aira Binti, 4.79
  - Ella Harris, 4.18
  - Ruchi Gupta, 4.12
  - Fangling, 4.05
  - Pooja Bhatia, 4.01
Group 7
  - Ciara Lim, 4.79
  - Griffin Yang, 4.18
  - Xanthe Scott, 4.12
  - Raj Patel, 4.05
  - Ritika Iyer, 4.01
Group 8
  - Claire Ng, 4.79
  - Rani Kaur, 4.18
  - Alexander Clark, 4.12
  - Shahril Bin Muni, 4.05
  - Zhou Min, 4.01
Group 9
  - Sari Rakhmawati, 4.79
  -

### 4.3.3) Diversifying Gender

With the remaining degree of freedom left from CGPA and School, Gender is diversified. To do this, we may assign the number of males & females that can be equally divided between each group first, then assign the remainder (r) of both male and females to the first groups sequentially (which we designed to always hold greater or equal number of students to the last few groups) to try to maintain the same ratio. 

This approach helps the individual group male to female ratio to tend to the population male to female ratio. 

In [72]:
def sortGender(studentData, n):
    total = len(studentData)
    r = total % n
    numGrp = total // n if (r <= total/(2 * n)) else total // n + 1
    grps = [[] for _ in range(numGrp)] 

    # Finding minimum
    totalMale = sum(1 for s in studentData if s['Gender'])
    mMin = totalMale // numGrp # Minimum possible number of males per group
    mRemainder = totalMale % numGrp # Remainder of males after distributing equally

    # Remainder males get populated into the first mRemainder groups
    mIdeal = [mMin] * numGrp
    for i in range(mRemainder):
        mIdeal[i] += 1
    mCurr = [0] * numGrp

    totalFemale = len(studentData) - totalMale
    fMin = totalFemale // numGrp # Minimum possible number of females per group
    fRemainder = totalFemale % numGrp # Remainder of females after distributing equally

    # Remainder females get populated into groups starting from the mRemainder'th index
    fIdeal = [fMin] * numGrp 
    for i in range (mRemainder, mRemainder + fRemainder):
        fIdeal[i % numGrp] += 1
    fCurr = [0] * numGrp

    for student in studentData:
        for g, grp in enumerate(grps):
            if student['Gender'] and mCurr[g] < mIdeal[g]:
                grps[g].append(student)
                mCurr[g] += 1
                break
            elif not student['Gender'] and fCurr[g] < fIdeal[g]:
                grps[g].append(student)
                fCurr[g] += 1
                break
                
    return grps

grps = sortGender(studentData, 11)
popMCount = sum(1 for student in studentData if student['Gender'])
print("M ", popMCount, " F ", len(studentData) - popMCount)
for i, grp in enumerate(grps, start=1):
    mCount = sum(1 for student in grp if student['Gender'])
    fCount = len(grp) - mCount
    print(f"Group {i} - {len(grp)}, ({mCount}, {fCount})")
    for s in grp:
        print(f"  - {s['Name']}, {'M' if s['Gender'] else 'F'}")

M  2759  F  3241
Group 1 - 12, (6, 6)
  - Hunter Green, M
  - Dante Ong, M
  - Nadia Binti Zain, M
  - Benjamin Hernandez, M
  - Suhana Bhat, F
  - Aira Binti, F
  - Ciara Lim, F
  - Claire Ng, F
  - Sari Rakhmawati, F
  - Maira Binti Rahman, F
  - Matthew Martinez, M
  - Faisal Bin Yusof, M
Group 2 - 12, (6, 6)
  - Yanzhi, F
  - Liyana Binti Mohd, F
  - Yongqiang, F
  - Nabihah Binti Rahman, F
  - Laras Wati, F
  - Ravi Mehta, M
  - Layla Torres, F
  - Yasir Ahmad, M
  - Ashok Nair, M
  - Jason Robinson, M
  - Asher Thompson, M
  - Mateo Huang, M
Group 3 - 12, (6, 6)
  - Hae-Jin Lee, F
  - Zadie Harris, F
  - Asha Mehta, F
  - Layla Reddy, F
  - Troy Wong, M
  - Nicolas Ramos, M
  - Uriah Howard, M
  - Harper Martinez, F
  - Nitesh Singh, M
  - Fauzi Bin Zain, M
  - Samantha Lee, F
  - Chandra Widyanto, M
Group 4 - 12, (6, 6)
  - Aaron Lee, M
  - Priyanka Singh, F
  - Olivia Johnson, F
  - Sadia Khan, F
  - Mi-kyung Han, F
  - Fatima Reddy, F
  - Luna Zhao, F
  - Tristan Smith, M
  - 

# Sorting Algorithm

In [ ]:
def distribute(studentData, n):

    total = len(studentData)
    r = total % n
    numGrp = total // n if (r <= total/(2 * n)) else total // n + 1
    grpSizes =  [n + 1 if i < r else n for i in range(numGrp)] if (r <= total/(2 * n)) else [r if i == total//n else n for i in range(numGrp)]
    grps = [[] for _ in range(numGrp)] 

    schoolCnt = {}
    for s in studentData:
        school = s['School']
        schoolCnt[school] = schoolCnt.get(school, 0) + 1
    sortedSchools = sorted(schoolCnt.keys(), key=lambda x: schoolCnt[x], reverse=True)

    totalMale = sum(1 for s in studentData if s['Gender'])
    mMin = totalMale // numGrp
    mRemainder = totalMale % numGrp
    mIdeal = [mMin + (1 if i < mRemainder else 0) for i in range(len(grpSizes))]
    mCurr = [0] * numGrp

    totalFemale = len(studentData) - totalMale
    fMin = totalFemale // numGrp
    fRemainder = totalFemale % numGrp
    fIdeal = [fMin + (1 if i < fRemainder else 0) for i in range(len(grpSizes))]
    fCurr = [0] * numGrp

    sortedStudents = sorted(studentData, key=lambda s: (sortedSchools.index(s['School']), -s['CGPA']))

    # Distribution
    grpIndex = 0
    for student in sortedStudents:
        for offset in range(numGrp):
            g = (grpIndex + offset) % numGrp
            if len(grps[g]) < grpSizes[g]:
                if student['Gender'] and mCurr[g] < mIdeal[g]:
                    grps[g].append(student)
                    mCurr[g] += 1
                    # print(f"append {student} to grp {g}")
                    break
                elif not student['Gender'] and fCurr[g] < fIdeal[g]:
                    grps[g].append(student)
                    fCurr[g] += 1
                    # print(f"append {student} to grp {g}")
                    break
            else:
                # if all gender quotas full, put in group with least size
                g = min(range(numGrp), key=lambda x: len(grps[x]))
                grps[g].append(student)
                if student['Gender']:
                    mCurr[g] += 1
                else:
                    fCurr[g] += 1
                # print(f"append {student} to grp {g} (backup)")

        grpIndex += 1

    return grps

In [39]:
n = 7

# Apply to each TG 
all_tgs = sorted(set(s['TG'] for s in studentData))
for tg in all_tgs:
    tgStuds = [s for s in studentData if s['TG'] == tg]
    totalStuds = len(tgStuds)

    # Get TG gender ratio
    mCount_tg = sum(1 for s in tgStuds if s['Gender'])
    fCount_tg = totalStuds - mCount_tg
    mRatio_tg = mCount_tg / totalStuds if totalStuds else 0
    fRatio_tg = 1 - mRatio_tg

    print(f"\nTutorial Group {tg}")
    print(f"Total students: {totalStuds}")
    print(f"Gender ratio: {mCount_tg}M / {fCount_tg}F "
          f"({mRatio_tg:.2%} male, {fRatio_tg:.2%} female)")

    # Get TG school ratio
    schoolDistTG = {}
    for s in tgStuds:
        schoolDistTG[s['School']] = schoolDistTG.get(s['School'], 0) + 1
    sortedSchoolsTG = sorted(schoolDistTG.keys(), key=lambda x: schoolDistTG[x], reverse=True)
    schoolSummary = ", ".join(f"{sch}: {schoolDistTG[sch]}" for sch in sortedSchoolsTG)
    print(f"School distribution: {schoolSummary}")

    # Distribute
    finalGrps = distribute(tgStuds, n)

    # Print results per grp
    totalInGrps = 0
    for i, g in enumerate(finalGrps, start=1):
        mCount = sum(1 for s in g if s['Gender'])
        fCount = len(g) - mCount
        totalInGrps += len(g)
        avgCGPA = sum(s['CGPA'] for s in g)/len(g)
        print(f"\nGroup {i} (size {len(g)}): Males: {mCount}, Females: {fCount}\nAvgCGPA = {avgCGPA}")
        for s in g:
            print(f"  - {s['Name']} ({s['School']}, {'M' if s['Gender'] else 'F'}, CGPA: {s['CGPA']})")

    print(f"\nTotal assigned in TG{tg}: {totalInGrps} / {totalStuds}")


# Getting it back into an excel file and things

In [40]:
testfile = open("testrecords.csv", "w")
fileHeader = fileHeader
fileData.insert(0, fileHeader) #puts back header line
fileData[1].append('')
fileData[1].append('AAAAAAAH')
print(fileData[0])
print(fileData[1])
for i in range(len(fileData)):  #for each list, join items by ','. Then prints each line into new excel sheet
    fileData[i] = ','.join(fileData[i])
    print(fileData[i], file = testfile, end = '\n')

['Tutorial Group', 'Student ID', 'School', 'Name', 'Gender', 'CGPA']
['G-1', '5002', 'CCDS', 'Aarav Singh', 'Male', '4.02', '', 'AAAAAAAH']


# Algorithm Evaluation

## Basic Factors
No majority of students of the same following factors:
1. School
2. Gender
3. Current CGPA

General flow:
TODO: insert flowchart
- For each tutorial group, check for a single factor first, to find the group with the least diversity
- Then rank them to see with tutorial group has a group with the least diversity

### Finding the diversity of a single factor of a single group
Required data: list of students in the group, required factor of the student
<br>
Output: The number for diversity
<br>
Start with finding the number of occurences of each factor

In [41]:
# Parameters:
#  factor: string of factor to check, same as the key in the dictionary
#  groupData: data needed as a list of dictionaries
# Return:
#  occurences: dictionary of {factor1: numOccurences, factor2: numOccurences ... } as string:int
def FindOccurencesInGroup(factor, groupData : list) -> dict:
    size = len(groupData)
    occurences = {} # Dictionary to keep track of occurences of each factor
    for i in groupData:
        factorOfStudent = i[factor]
        # If factor already exists in occurences, add 1 to it, otherwise create it and set to 1
        if factorOfStudent in occurences.keys():
            occurences[factorOfStudent] += 1
        else:
            occurences[factorOfStudent] = 1
    return occurences

In [42]:
testGroupData = [
    {"Student ID": 71, "TG" : "G-1", "School" : "SoH", "Name" : "Savannah Taylor", "Gender" : False, "CGPA" : 4.19},
    {"Student ID": 75, "TG" : "G-1", "School" : "CCDS", "Name" : "Sakina Ahmed", "Gender" : False, "CGPA" : 4.03},
    {"Student ID": 162, "TG" : "G-1", "School" : "SSS", "Name" : "Lila Patel", "Gender" : False, "CGPA" : 4.07},
    {"Student ID": 235, "TG" : "G-1", "School" : "CCDS", "Name" : "Ming Zhang", "Gender" : True, "CGPA" : 4.06},
    {"Student ID": 288, "TG" : "G-1", "School" : "CoB (NBS)", "Name" : "Ajay Verma", "Gender" : True, "CGPA" : 4.01}
]
print(FindOccurencesInGroup("School", testGroupData))
print(FindOccurencesInGroup("Gender", testGroupData))
print(FindOccurencesInGroup("CGPA", testGroupData))

{'SoH': 1, 'CCDS': 2, 'SSS': 1, 'CoB (NBS)': 1}
{False: 3, True: 2}
{4.19: 1, 4.03: 1, 4.07: 1, 4.06: 1, 4.01: 1}


#### Then find the most number of occurences out of the total students

In [43]:
# Summary: returns the key and value of the key with the largest value
# Parameters:
#  dictionary: the dict to find the max of
# Return:
#  key : list, values : int, where key is the key with highest value
def FindMaxInDict(dictionary: dict) -> (list, int):
    maxKeys = []
    maxVal = 0
    for key, value in dictionary.items():
        # update maxKeys and maxVal whenever a new value >= current maxVal
        if value > maxVal:
            maxKeys = [key]
            maxVal = value
        elif value == maxVal:
            maxKeys.append(key)
    return maxKeys, maxVal

In [44]:
print(FindMaxInDict(FindOccurencesInGroup("School", testGroupData)))
print(FindMaxInDict(FindOccurencesInGroup("Gender", testGroupData)))
print(FindMaxInDict(FindOccurencesInGroup("CGPA", testGroupData)))

(['CCDS'], 2)
([False], 3)
([4.19, 4.03, 4.07, 4.06, 4.01], 1)


## CGPA case
But this does not work for CGPA, for that we want to find the mean, range, and midrange instead
If we use only mean, a majority of high CGPA with a very low CGPA will result in a medium CGPA, this can be solved by looking at midrange, where a high difference between midrange and mean will show that this is happening
However, even with these two, a group will all medium CGPAs will also result in medium mean, and low midrange difference, therefore we add range to determine that this is happening if the range is small.
In conclusion, the result we are looking for is one with an medium mean, high range, and low difference between midrange and mean.
### Finding mean, midrange, and range

In [45]:
def Mean(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    return sum(nums) / len(nums)

def Range(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    return max(nums) - min(nums)

def Midrange(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    return (max(nums) + min(nums)) / 2

def Median(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    # TODO: replace with another self-made sort cherry pls
    nums.sort()
    total = len(nums)
    mid = total // 2
    if total % 2:
        return (nums[mid] + nums[mid+1]) / 2
    return nums[mid]

In [46]:
testList = [4.19, 4.03, 4.07, 4.06, 4.01]
print(Mean(testList))
print(Range(testList))
print(Midrange(testList))
print(Median(testList))

4.072
0.1800000000000006
4.1
4.0649999999999995


## Calculating for CGPA

In [47]:
# Parameters:
#  medianCGPA: median CGPA to use for calculation (likely the tutorial grp's)
#  CGPARange: Range of CGPA to use for calculation (likely the tutorial grp's)
#  groupData: list of studentData to find CGPA diversity (likely a group within the tut grp)
# Return:
#  score: overall score of CGPA
#  cGPADetails: dictionary of {factor: percent} the factor "balance" students with higher CGPA - students with lower CGPA than median
def CalculateCGPADiversity(medianCGPA : float, CGPARange : float, *groupData) -> (int, dict):
    if len(groupData) == 1 and type(groupData[0]) is list:
        groupData = groupData[0]

    cGPADetails = {}
    # Calculate higher and lowers
    lower = higher = 0
    cGPAs = ExtractCGPA(groupData)
    for gpa in cGPAs:
        if gpa < medianCGPA:
            lower += 1
        elif gpa > medianCGPA:
            higher += 1
    cGPADetails["balance"] = higher - lower

    # Calculate the score for each factor
    cGPADetails["mean"] = Mean(cGPAs)
    cGPADetails["range"] = Range(cGPAs)
    cGPADetails["midrange"] = Midrange(cGPAs)

    # Score 0 to 100, 100 is most diverse possible
    # Usage of 5 as is prevalent as it is the max CGPA and max difference between lowest and highest CGPA
    
    # vv How close mean is to the median vv
    cGPADetails["meanScore"] = (5 - abs(cGPADetails["mean"] - medianCGPA)) * 20
    # vv Ratio of grp's range and tutorial grp's range vv
    cGPADetails["rangeScore"] = cGPADetails["range"] / CGPARange * 100
    # vv How far midrange is from mean vv   most diverse ==> mean of x, midrange of x
    cGPADetails["midrangeScore"] = (1 - abs(cGPADetails["mean"] - cGPADetails["midrange"])) * 100
    # vv Lowest: score = 0 when abs(balance) = Num of grp members, Highest: score = 100 when balance = 0 vv 
    cGPADetails["balanceScore"] = (len(groupData) - abs(cGPADetails["balance"])) / len(groupData) * 100
    
    # Overall score out of 100
    # Weightages for what we consider more important
    W_RANGE = 1
    W_MIDRANGE = 1
    W_MEAN = 2
    W_BALANCE = 2
    
    score = (cGPADetails["meanScore"] * W_MEAN\
             + cGPADetails["rangeScore"] * W_RANGE\
             + cGPADetails["midrangeScore"] * W_MIDRANGE\
             + cGPADetails["balanceScore"] * W_BALANCE) / sum([W_MEAN, W_RANGE, W_MIDRANGE, W_BALANCE])
    return score, cGPADetails

# Returns a list of CGPA of the students in groupData
def ExtractCGPA(*groupData) -> list:
    if len(groupData) == 1 and type(groupData[0]) is list:
        groupData = groupData[0]

    cGPAs = []
    for i in groupData:
        cGPAs.append(i["CGPA"])
    return cGPAs

# Returns a list of CGPA of the students in groupData
def TutGrpExtractCGPA(tutGroupData) -> list:
    cGPAs = []
    for i in tutGroupData:
        cGPAs.extend(ExtractCGPA(i))
    return cGPAs

In [48]:
#TODO : revise the calculation of balance factor by taking into account buckets rather than median
testGroupData = finalGrps
medianCGPA = Median(TutGrpExtractCGPA(testGroupData))
rangeCGPA = Range(TutGrpExtractCGPA(testGroupData))
print(medianCGPA)
for i in range(len(testGroupData)):
    print(f"test group: {i}")
    PrintDictListNice(testGroupData[i])
    print()
    result = CalculateCGPADiversity(medianCGPA, rangeCGPA, testGroupData[i])
    print("Score: ", result[0])
    PrintDictListNice([result[1]])
# print(testGroupData)


4.04
test group: 0
ID: 3721              TG: 120               School: CoB (NBS)         Name: Samantha Lee      Gender: False             CGPA: 4.520-            
ID: 5220              TG: 120               School: CoB (NBS)         Name: Rishi Singh       Gender: True              CGPA: 4.080-            
ID: 3222              TG: 120               School: CCDS              Name: Kang Lei          Gender: False             CGPA: 4.040-            
ID: 5835              TG: 120               School: MAE               Name: Samuel Cook       Gender: True              CGPA: 4.020-            
ID: 822               TG: 120               School: CCEB              Name: Riku Tanaka       Gender: False             CGPA: 4.030-            
ID: 4631              TG: 120               School: EEE               Name: Samantha Patel    Gender: False             CGPA: 4.000-            
ID: 2420              TG: 120               School: CEE               Name: Faisal Bin Yusof  Gender: True     

## Calculating the factors for each group
Combine all the previous functions together to obtain the data for the least diverse of each factor and which group(s) they belong to

In [73]:
# Parameters:
#  factors: list of strings of factor to check, same as the key in the dictionary
#  groupsDatas: List of datas needed as a list of dictionaries
# Return:
#  leastDiverse: dictionary of {factor1: occurencesPercent, factor2: occurencesPercent ... } as string:int
#  grpIdx: dictionary of the grps that have the least diversity for the factor {factor1: groups}
def FindLeastDiverseInTutorialGroup(factors, *groupsDatas) -> (dict, list):
    if len(groupsDatas) == 1 and type(groupsDatas[0]) is list:
        groupsDatas = groupsDatas[0]

    # Get data required for calculating diversity score for CGPA
    allCGPA = TutGrpExtractCGPA(testGroupData)
    tutMedianCGPA = Median(allCGPA)
    tutRangeCGPA = Range(allCGPA)
    
    grpIdx = {}
    leastDiverse = {}
    for factor in factors:
        # Set default value to replace
        leastDiverseOccurence : float = ("factorResult", 0) # % of highest occurence, i.e. 90% of students are from ccds
        currentGrpIdx = 0
        for group in groupsDatas:
            # Special CGPA case
            if factor == "CGPA":
                result = CalculateCGPADiversity(tutMedianCGPA, tutRangeCGPA, group) # result is (score : float, details : dictionary)
                occurence = [["CGPA score"], result[0]]
                currentPercent = occurence[1]
            else:
                occurence = FindMaxInDict(FindOccurencesInGroup(factor, group)) # result is ([least diverse factors(i.e. CCDS, False)], occurence count)
                currentPercent = occurence[1] / len(group)
            # Less diverse so replace
            if currentPercent > leastDiverseOccurence[1]\
            or (factor == "CGPA" and currentPercent < leastDiverseOccurence[1]): # Special CGPA case where currentPercent is CGPA diversity score, so want the lowest
                leastDiverseOccurence = (occurence[0], currentPercent)
                grpIdx[factor] = [currentGrpIdx]
            # Same diversity so append
            elif currentPercent == leastDiverseOccurence[1]:
                leastDiverseOccurence = (list(leastDiverseOccurence[0] + occurence[0]), currentPercent)
                grpIdx[factor].append(currentGrpIdx)
            currentGrpIdx += 1
        leastDiverse[factor] = leastDiverseOccurence
    return leastDiverse, grpIdx

## Put it into a function that prints out all the data

In [74]:
# Parameters:
#  groupsData: List of datas needed as a list of dictionaries -> Tutorial group
# Return: (same as function FindLeastDiverseInTutorialGroup)
#  leastDiverse: dictionary of {factor1: occurencesPercent, factor2: occurencesPercent ... } as string:int
#  grpIdx: dictionary of the grps that have the least diversity for the factor {factor1: groups}
def PrintResultForTutorialGroup(groupData):
    allCGPA = TutGrpExtractCGPA(groupData)
    tutMedianCGPA = Median(allCGPA)
    tutRangeCGPA = Range(allCGPA)
    for i in range(len(groupData)):
        print(f"test group: {i}")
        PrintDictListNice(groupData[i])
        print()
        result = CalculateCGPADiversity(medianCGPA, rangeCGPA, groupData[i])
        print("CGPA Score: ", result[0], end="\n\n")
    # print(groupData)
    # print(FindLeastDiverseInTutorialGroup(["School", "Gender", "CGPA"], groupData))
    return PrintDictListNice(FindLeastDiverseInTutorialGroup(["School", "Gender", "CGPA"], groupData))
PrintResultForTutorialGroup(finalGrps)

test group: 0
ID: 3721              TG: 120               School: CoB (NBS)         Name: Samantha Lee      Gender: False             CGPA: 4.520-            
ID: 5220              TG: 120               School: CoB (NBS)         Name: Rishi Singh       Gender: True              CGPA: 4.080-            
ID: 3222              TG: 120               School: CCDS              Name: Kang Lei          Gender: False             CGPA: 4.040-            
ID: 5835              TG: 120               School: MAE               Name: Samuel Cook       Gender: True              CGPA: 4.020-            
ID: 822               TG: 120               School: CCEB              Name: Riku Tanaka       Gender: False             CGPA: 4.030-            
ID: 4631              TG: 120               School: EEE               Name: Samantha Patel    Gender: False             CGPA: 4.000-            
ID: 2420              TG: 120               School: CEE               Name: Faisal Bin Yusof  Gender: True          

In [51]:
# Weights
CRITERA_WEIGHTS = {"CGPA": 100, "SCHOOL": 100, "GENDER": 100}

# Graphical plots and things

# Challenges and Issues

- 

# Conclusion

# Debug test things or test cases?
doubt they will give marks for test cases tho, prob just leave this for our own use and remove before submission
Highlight

In [52]:
def PrintDictListNice(dictionaryList, spacing : int = 18):
    for i in dictionaryList:
        for k,v in i.items():
            contentLength = 10
            if type(v) is float:
                v = f"{v:.3f}-"
            if type(v) is int:
                contentLength = log10(v) + 1
            else:
                contentLength = len(str(v))
            # elif type(v) is bool:
            #     contentLength = len(str(v))
            print(f"{k}: {v}{' ' * max(spacing - contentLength, 1)}", end = '')
        print()
def log10(num):
    count = 0
    while num >= 10:
        num /= 10
        count += 1
    return count

In [53]:
print(log10(1))
print(log10(20))
print(log10(34))
print(log10(64280))
d = {"test" : 4.236752387546, "a" : "g"}
PrintDictListNice([d])

0
1
1
4
test: 4.237-            a: g                 
